In [1]:
import numpy as np
import pandas as pd
import h5py

import matplotlib.pyplot as plt

from tqdm import tqdm
from copy import deepcopy
from joblib import Parallel, delayed

%load_ext autoreload
%autoreload 2

from galquench.utils import *


In [2]:
folder = "/Users/richard/Projects/galquench/data/tng100/"


subhalos_file = {"basePath": folder + "groupcats",
                 "snapNum": 99,
                 "fields": ["SubhaloMassType", "SubhaloStellarPhotometricsRad"]}

supplementary_files = [
    {"file" : folder + "star_formation_rates.hdf5",
     "subfindID": "SubfindID",
     "snapshot_number": 99,
     "keys": ["SFR_MsunPerYrs_in_all_1000Myrs"]},
    {"file" : folder + "hih2_galaxy_099.hdf5",
     "subfindID": "id_subhalo",
     "keys": ["m_neutral_H"]}
    ]

In [10]:
subhalos = loadSubhalos(**subhalos_file)
supplementary_cats = [None] * len(supplementary_files)
for i, f in enumerate(supplementary_files):
    supplementary_cats[i] = read_supplementary(**f)


In [11]:
column_mapping = {"SubhaloMassType": (1, 4)}

for cat in [subhalos] + supplementary_cats:
    unpack_catalog_columns(cat, column_mapping)

In [12]:
match_subfind(supplementary_cats, subhalos["count"])
arr = merge_subhalos_with_supplementary(subhalos, supplementary_cats)


In [60]:
path = '/Users/richard/Projects/galquench/data/tng100/stellar_circs.hdf5'

X = read_supplementary(path, "SubfindID", keys="SpecificAngMom", snapshot_number=99)
X

{'subfindID': array([      0,       1,       2, ..., 1036877, 1057646, 1189263]),
 'SpecificAngMom': array([8029.7563  , 1019.8987  , 3793.828   , ...,   12.561851,
          19.321045,   11.684096], dtype=float32)}

In [59]:
path = '/Users/richard/Projects/galquench/data/tng100/kinematic_decomposition_099.hdf5'

X = read_supplementary(path, "SubhaloID")
X

{'subfindID': array([    10,     12,     16, ..., 590461, 599631, 609986]),
 'Header': <HDF5 group "/Header" (0 members)>,
 'class1_1re': <HDF5 group "/class1_1re" (8 members)>,
 'class1_3re': <HDF5 group "/class1_3re" (8 members)>,
 'class1_allstars': <HDF5 group "/class1_allstars" (8 members)>,
 'class2_1re': <HDF5 group "/class2_1re" (8 members)>,
 'class2_3re': <HDF5 group "/class2_3re" (8 members)>,
 'class2_allstars': <HDF5 group "/class2_allstars" (8 members)>}

In [54]:
X

{'subfindID': array([    10,     12,     16, ..., 590461, 599631, 609986]),
 'Header': array([], dtype=float64),
 'class1_1re': array(['Bulge', 'ColdDisk', 'Disks', 'DiskyBulge', 'Halo', 'Spheroids',
        'StellarMass', 'WarmDisk'], dtype='<U11'),
 'class1_3re': array(['Bulge', 'ColdDisk', 'Disks', 'DiskyBulge', 'Halo', 'Spheroids',
        'StellarMass', 'WarmDisk'], dtype='<U11'),
 'class1_allstars': array(['Bulge', 'ColdDisk', 'Disks', 'DiskyBulge', 'Halo', 'Spheroids',
        'StellarMass', 'WarmDisk'], dtype='<U11'),
 'class2_1re': array(['Bulge', 'ColdDisk', 'Disks', 'DiskyBulge', 'Halo', 'Spheroids',
        'StellarMass', 'WarmDisk'], dtype='<U11'),
 'class2_3re': array(['Bulge', 'ColdDisk', 'Disks', 'DiskyBulge', 'Halo', 'Spheroids',
        'StellarMass', 'WarmDisk'], dtype='<U11'),
 'class2_allstars': array(['Bulge', 'ColdDisk', 'Disks', 'DiskyBulge', 'Halo', 'Spheroids',
        'StellarMass', 'WarmDisk'], dtype='<U11')}

In [52]:
X = h5py.File('/Users/richard/Projects/galquench/data/tng100/kinematic_decomposition_099.hdf5', 'r')
X.keys()

<KeysViewHDF5 ['Header', 'SubhaloID', 'class1_1re', 'class1_3re', 'class1_allstars', 'class2_1re', 'class2_3re', 'class2_allstars']>

In [44]:
type(X)

h5py._hl.files.File

In [38]:
X.keys()

<KeysViewHDF5 ['Header', 'SubhaloID', 'class1_1re', 'class1_3re', 'class1_allstars', 'class2_1re', 'class2_3re', 'class2_allstars']>

In [41]:
X["class1_3re"].keys()

<KeysViewHDF5 ['Bulge', 'ColdDisk', 'Disks', 'DiskyBulge', 'Halo', 'Spheroids', 'StellarMass', 'WarmDisk']>

In [30]:
X["subfindID"].dtype

dtype('int64')

In [14]:
X["id_subhalo"]

KeyError: 'id_subhalo'

In [5]:
halos = parse_subhalos0(subhalos0, halo_legend)
gals = parse_subhalos0(subhalos0, galaxy_legend)

# stellar_circs = h5py.File('../data/IllustrisTNG/stellar_circs.hdf5', 'r')

N = halos.size

out = {}
for par in halos.dtype.names:
    out.update({'H_' + par: halos[par]})

for par in gals.dtype.names:
    out.update({'G_' + par: gals[par]})    

    
concentration = pd.read_hdf('../data/IllustrisTNG/christmas_data/TNG100-1_GalHalo_Subfind_snap99.hdf5')
concentration[concentration['EndState'].values != 'CHeck100DM'] # Take only ones that have converged
# From concentration we want
for par in ['Central', 'Rsubfind_DM', 'Msubfind_DM', 'Vmax', 'cvir', 's_DM', 'q_DM', 'Spin_Bullock', 'Mvir_host', 'r_s_fit']:
    x = np.full(N, np.nan)
    x[concentration['SubhaloID'].values] = concentration[par]
    out.update({'H_' + par: x})
    
out['H_Rsubfind'] = out.pop('H_Rsubfind_DM')
out['H_Msubfind'] = out.pop('H_Msubfind_DM')

for par in ['Mgas', 'Mstar', 'Rhalf_star', 's_star', 'q_star', 'Spin_Bullock_star']:
    x = np.full(N, np.nan)
    x[concentration['SubhaloID'].values] = concentration[par]
    out.update({'G_' + par: x})
    
concentration = pd.read_hdf('../data/IllustrisTNG/christmas_data/TNG100-1_GalHalo_snap99.hdf5')
concentration[concentration['EndState'].values != 'CHeck100DM'] # Take only ones that have converged
# From concentration we want
for par in ['frac_rho_vir', 'Rvir']:
    x = np.full(N, np.nan)
    x[concentration['SubhaloID'].values] = concentration[par]
    out.update({'H_' + par: x})
    

level = out.pop('H_Central')
level[level == 0] = 2 # Make 0 to 2 for subhals
out.update({'H_level': level})

# x = np.full(N, np.nan)
# x[stellar_circs['Snapshot_99']['SubfindID'][:]] = stellar_circs['Snapshot_99']['SpecificAngMom']
# out.update({'G_' + 'SpecificAngMom': x})

# x = np.full(N, np.nan)
# x[stellar_circs['Snapshot_99']['SubfindID'][:]] = stellar_circs['Snapshot_99']['MassTensorEigenVals'][:,0]
# out.update({'G_' + 'sha': x})

# x = np.full(N, np.nan)
# x[stellar_circs['Snapshot_99']['SubfindID'][:]] = stellar_circs['Snapshot_99']['MassTensorEigenVals'][:, 1]
# out.update({'G_' + 'shb': x})

# x = np.full(N, np.nan)
# x[stellar_circs['Snapshot_99']['SubfindID'][:]] = stellar_circs['Snapshot_99']['MassTensorEigenVals'][:, 2]
# out.update({'G_' + 'shc': x})
    
# out.update({'G_ca_ratio': out['G_shc'] / out['G_sha']})
# out.update({'G_ba_ratio': out['G_shb'] / out['G_sha']})


neutral_hydrogen = h5py.File('../data/IllustrisTNG/hih2_galaxy_099.hdf5')
x = np.full(N, np.nan)
x[neutral_hydrogen['id_subhalo'][:].astype(int)] = neutral_hydrogen['m_neutral_H']
out.update({'G_' + 'm_neutral_H': x})

sfrs = h5py.File('../data/IllustrisTNG/star_formation_rates.hdf5')
x = np.full(N, np.nan)
x[sfrs['Snapshot_99']['SubfindID'][:]] = sfrs['Snapshot_99']['SFR_MsunPerYrs_in_all_200Myrs']
out.update({'G_' + 'SFR_MsunPerYrs_in_all_200Myrs': x})


# out.update({'G_Reff_r': h5py.File('../data/IllustrisTNG/stellar_sizes_099.hdf5')['Rhalf'][0, -3, :]})


# level = out.pop('H_Central')
# level[level == 0] = 2
# out.update({'H_level': level})


# pc = 30856780000000000
# vvir = np.sqrt(6.674e-11 * 10**(out['H_Mvir']) * 1.989e30 / (out['H_Rvir']* 1e3 * pc))*1e-3
# out.update({'tot_spin': np.sqrt(out['H_spinx']**2 + out['H_spiny']**2 + out['H_spinz']**2) / vvir / out['H_Rvir']})

# N = halos.size

# out.update({'G_mgal_and_mgas': out['G_mgal'] + out['G_mgas']})
# out.update({'G_mgal_and_m_neutral_H': out['G_mgal'] + out['G_m_neutral_H']})

cat = np.full(N, np.nan, dtype={'names': [key for key in out.keys()],
                                'formats': [type(val[0]) for val in out.values()]})





for key in out.keys():
    cat[key] = out[key]
    
del out, x, halos, gals

In [ ]:
m = cat['H_flag']

m = m & (~np.isnan(cat['H_cvir']))
m = m & (cat['H_Msubfind'] > 10) & (cat['G_Mstar'] > 6)
m = m & (cat['G_Mgas'] > 0) & (cat['G_m_neutral_H'] > 0)
m = m & (cat['G_sfr_instant'] > 0)

cat_final = cat[m]

cat_final['H_Msubfind'] = 10**cat_final['H_Msubfind']
cat_final['H_Mvir_host'] = 10**cat_final['H_Mvir_host']
cat_final['G_Mgas'] = 10**cat_final['G_Mgas']
cat_final['G_Mstar'] = 10**cat_final['G_Mstar']

In [ ]:
np.save('../data/IllustrisTNG/TNGmatch.npy', cat_final)

In [ ]:
cat_final = np.load('../data/IllustrisTNG/TNGmatch.npy')

HAGN = np.load('../data/HAGN/extracted/matched_objects.npy')

G_Rhalfmass: Comoving radius containing half of the mass of this Subhalo split by Type (SubhaloMassType).

G_Reff: Radius at which the surface brightness profile (computed from all member stellar particles) drops below the limit of 20.7 mag arcsec−2−2 in the K band (in comoving units).

G_Reff_r : half-light radiur in the r-band 

# Reff vs M*

In [ ]:
level = 1

m = (cat_final['H_level'] == level) # & (cat_final['G_m_neutral_H'] > 10**7) 
m = m & (cat_final['G_Rhalfmass'] > 0)
m = m & (cat_final['H_Mvir'] > 10**10)
m = m & (cat_final['G_m_neutral_H'] > 0)

m2 = HAGN['H_level'] == level


plt.figure()


# plt.scatter(cat_final['H_Mvir'][m], cat_final['G_mgal_and_mgas'][m], s=1, label='TNG')
plt.scatter(cat_final['H_Mvir'][m], cat_final['G_mgal_and_m_neutral_H'][m], s=1, label='TNG')
# plt.scatter(cat_final['H_Mvir'][m], cat_final['G_mgal'][m], s=1, label='TNG')
# plt.scatter(cat_final['H_Mvir'][m], cat_final['G_mgas'][m], s=1, label='TNG')
# plt.scatter(cat_final['G_mgal'][m], cat_final['G_Reff'][m]/1e5, s=1, label='TNG')
# plt.scatter(HAGN['H_mvir'][m2], HAGN['G_Reff'][m2], s=1, label='HAGN')


plt.xlabel("Mgal")
plt.ylabel("Reff")

plt.legend()
plt.yscale('log')
plt.xscale('log')

plt.show()

# Reff_r vs Mgal

In [ ]:
level = 1

m = (cat_final['H_level'] == level) # & (cat_final['G_m_neutral_H'] > 10**7) 
# m = m & (cat_final['G_Reff_r'] > 0)
m = m & (cat_final['H_Mvir'] > 10**10)

m2 = HAGN['H_level'] == level


plt.figure()


plt.scatter(HAGN['H_mvir'][m2], HAGN['G_Reff'][m2], s=1, label='HAGN')
plt.scatter(cat_final['H_Mvir'][m], cat_final['G_Rhalfmass'][m]/1e4, s=1, label='TNG')



plt.xlabel("Mvir")
plt.ylabel("Reff")

plt.legend()
plt.yscale('log')
plt.xscale('log')

plt.savefig('../plots/Reff.png', dpi=240)
plt.show()

# M* vs Mvir

In [ ]:
level = 1

m = (cat_final['H_level'] == level) # & (cat_final['G_m_neutral_H'] > 10**7) 
m = m & (cat_final['G_mgal'] > 0)
m = m & (cat_final['H_Mvir'] > 10**10)

m2 = HAGN['H_level'] == level


plt.figure()



plt.scatter(HAGN['H_mvir'][m2], HAGN['G_mgal'][m2], s=1, label='HAGN')
plt.scatter(cat_final['H_Mvir'][m], cat_final['G_mgal'][m], s=1, label='TNG')


plt.xlabel("Mvir")
plt.ylabel("Mgal")

plt.legend()
plt.yscale('log')
plt.xscale('log')

plt.show()

# HAGN spins vs size

In [ ]:
level = 2

m = (cat_final['H_level'] == level) # & (cat_final['G_m_neutral_H'] > 10**7) 
m = m & (cat_final['G_Reff'] > 0)
m = m & (cat_final['H_Mvir'] > 10**10) #& (cat_final['H_Mvir'] < 10**12)

m2 = HAGN['H_level'] == level


plt.figure()



# plt.scatter(HAGN['H_spin'][m2], HAGN['G_Reff'][m2]/HAGN['G_rvir'][m2], s=1, label='HAGN')
plt.scatter(cat_final['H_Msubfind'][m], cat_final['H_Rvir'][m], s=1, label='HAGN')
# plt.scatter(cat_final['H_Spin_Bullock'][m], cat_final['G_Rhalfmass'][m] / cat_final['H_Rvir'][m], s=1, label='TNG')


plt.xlabel("halo spin")
plt.ylabel("Reff/rvir")

plt.legend()
plt.yscale('log')
plt.xscale('log')

plt.show()

# Reff_r spins

In [ ]:
level = 1

m = (cat_final['H_level'] == level) # & (cat_final['G_m_neutral_H'] > 10**7) 
m = m & (cat_final['G_Rhalfmass'] > 0)
m = m & (cat_final['H_Mvir'] > 10**10)

m2 = HAGN['H_level'] == level


plt.figure()



# plt.scatter(HAGN['H_spin'][m2], HAGN['G_Reff'][m2]/HAGN['G_rvir'][m2], s=1, label='HAGN')
plt.scatter(cat_final['H_Spin_Bullock'][m], cat_final['G_Rhalfmass'][m] / cat_final['H_Rvir'][m], s=1, label='TNG')


plt.xlabel("halo spin")
plt.ylabel("Reff/rvir")

plt.legend()
plt.yscale('log')
plt.xscale('log')

plt.show()

In [ ]:
HAGN['H_ba_ratio']

In [ ]:
m = cat_final['H_level'] == 2
m2 = HAGN['H_level'] == 1


plt.figure()

# plt.scatter(HAGN['H_ca_ratio'][m2], HAGN['H_ba_ratio'][m2], s=0.5, label='HAGN', rasterized=True)
plt.scatter(cat_final['H_Mvir'][m], cat_final['H_cvir'][m], s=0.5, label='TNG', rasterized=True)

# plt.xlabel('c/a')
# plt.ylabel('b/a')

plt.xscale('log')
plt.yscale('log')

plt.legend()
# plt.savefig('../plots/ax_ratios.pdf')

plt.show()

In [ ]:
HAGN.dtype.names

In [ ]:
np.percentile(np.log10(cat_final['G_mgas'][m]), 16)

In [ ]:
m = cat_final['H_level'] == 2

# m = (cat_final['H_Mvir'] > 10**11.75) & (cat_final['H_Mvir'] <= 10**12.25)

plt.figure()

plt.scatter(cat_final['H_Mvir'][m], cat_final['H_Rvir'][m], s=1)
# plt.hist(np.log10(cat_final['G_mgas'][m]), bins='auto')


# m2 = (HAGN['H_level'] == 1)# & (HAGN['G_mstarform'] > 0)

# plt.scatter(HAGN['H_mvir'][m2], HAGN['G_Reff'][m2]*5000, s=1)

plt.yscale('log')
plt.xscale('log')
plt.title('Satellites only')

plt.xlabel('Mvir')
plt.ylabel('Rvir')

plt.savefig('Rvir.png', dpi=300)

# 
plt.show()

In [ ]:
HAGN = np.load('../data/HAGN/extracted/matched_objects.npy')


In [ ]:
m = HAGN['H_level'] == 2

In [ ]:
plt.figure()
# plt.scatter(HAGN['H_spin'][m], HAGN['G_Reff'][m], s=1)
plt.scatter(HAGN['H_mvir'][m], HAGN['G_Reff'][m], s=1)

plt.yscale('log')
plt.xscale('log')
plt.show()

In [ ]:
HAGN